# **Filogenia**

A filogenia é um campo que estuda as relações evolutivas entre organismos. Existem várias técnicas e métodos para a construção de árvores filogenéticas, e a escolha dependerá do tipo de dados disponíveis e das perguntas a responder.

Para análise filogenética utilizando o algoritmo **Needleman-Wunsch** devemos garantir estas etapas:

- **Loop de Alinhamento**:

    *Este loop deve invocar o algoritmo ***Needleman-Wunsch*** para cada par de sequências e, em seguida, atualizar a matriz ou estrutura de dados que armazena as pontuações ou distâncias entre as sequências.*

- **Matriz com um Dado Score**:

    *A matriz pode ser inicializada com zeros, e o loop de alinhamento será responsável por preenchê-la com os valores de pontuação calculados pelo algoritmo ***Needleman-Wunsch.***

- **Cálculo e Agrupamento Progressivo**:

    *Após realizar os alinhamentos progressivos e preencher a matriz de distâncias, é necessário implementar um método para calcular as distâncias progressivas e agrupar as sequências de acordo com essas distâncias. ***nw_ciclico***

- **Profit**:

    *O "Profit" refere-se à avaliação da qualidade do agrupamento ou à construção da árvore filogenética. Isso pode envolver a análise da estabilidade do agrupamento, a avaliação do bootstrap, ou a construção da árvore final.*

Para iniciar a **análise filogenética** necessitamos de fazer algumas modificações aos algoritmos desenvolvidos previamente apenas para o alinhamento:

In [110]:
def needleman_wunsch(seq1, seq2, score_subst, score_space):
    """
    Executa o algoritmo Needleman-Wunsch para alinhamento global de duas sequências.

    Parâmetros:
    - seq1 (str): Primeira sequência de entrada.
    - seq2 (str): Segunda sequência de entrada.
    - score_subst (int): Pontuação para correspondência ou penalidade para substituição.
    - score_space (int): Penalidade para espaços (inserção ou exclusão).

    Retorna:
    Tuple[str, str, int]: Uma tupla contendo as sequências alinhadas, onde o primeiro elemento é a sequência alinhada da primeira entrada, o segundo elemento é a sequência alinhada da segunda entrada, e o terceiro elemento é o escore do alinhamento.

    Modificações/alterações feitas ao algoritmo base:
    - Adição de Pontuações de Espaço: As pontuações de espaço são aplicadas durante a inicialização da primeira coluna e linha da matriz de pontuação.
    - Aprimoramento no Traceback: Durante o traceback, o código verifica se o caminho escolhido é uma correspondência (M), exclusão (D), ou inserção (I).
    - Escolha da Melhor Opção: O código seleciona a melhor opção entre correspondência, exclusão e inserção, usando a função max e atribuindo à posição atual na matriz de pontuação.
    - Scores: Os escores são calculados e armazenados em uma matriz de pontuação.
    """
    len_1, len_2 = len(seq1), len(seq2)
     
    # Inicializar a matriz de pontuação e a matriz de traceback
    score = [[0] * (len_2 + 1) for _ in range(len_1 + 1)]
    traceback = [[''] * (len_2 + 1) for _ in range(len_1 + 1)]
     
    # Inicializar a primeira coluna e linha com penalidades de espaço
    for i in range(len_1 + 1):
        score[i][0]     = i * score_space  # Penalidade de espaço na coluna
        traceback[i][0] = 'D'               # Marcador de Deleção (espaço)
    for j in range(len_2 + 1):
        score[0][j]     = j * score_space  # Penalidade de espaço na linha
        traceback[0][j] = 'I'               # Marcador de Inserção (espaço)

    # Preencher a matriz de pontuação e a matriz de traceback
    for i in range(1, len_1 + 1):
        for j in range(1, len_2 + 1):
            match = score[i-1][j-1] + (score_subst if seq1[i-1] == seq2[j-1] else -score_subst)  # Pontuação da correspondência ou penalidade de substituição
            delete = score[i-1][j] + score_space  # Penalidade de espaço para deleção
            insert = score[i][j-1] + score_space  # Penalidade de espaço para inserção

            choices = [match, delete, insert]
            best_choice = max(choices)

            score[i][j] = best_choice
            traceback[i][j] = 'M' if best_choice == match else ('D' if best_choice == delete else 'I')  # Escolha do caminho ótimo

    # Realizar o traceback para obter as sequências alinhadas
    aligned_seq1, aligned_seq2 = '', ''
    i, j = len_1, len_2
    while i > 0 and j > 0:
        if traceback[i][j] == 'M':
            aligned_seq1 = seq1[i-1] + aligned_seq1
            aligned_seq2 = seq2[j-1] + aligned_seq2
            i -= 1
            j -= 1
        elif traceback[i][j] == 'D':
            aligned_seq1 = seq1[i-1] + aligned_seq1
            aligned_seq2 = '-' + aligned_seq2
            i -= 1
        elif traceback[i][j] == 'I':
            aligned_seq1 = '-' + aligned_seq1
            aligned_seq2 = seq2[j-1] + aligned_seq2
            j -= 1

    return aligned_seq1, aligned_seq2, score[len_1][len_2]

def aprimorar_seq(string_1, length):
    """
    Adiciona espaços (lacunas) ao final da sequência para atingir o comprimento desejado.

    Parâmetros:
    - string_1 (str): A sequência de entrada.
    - length (int): O comprimento desejado para a sequência estendida.

    Retorna:
    str: A sequência estendida até o comprimento especificado.

    Modificações/alterações feitas ao algoritmo base:
    - Adição de Parâmetro de Comprimento (length): Agora a função aceita um comprimento desejado como parâmetro, garantindo que a sequência seja estendida até o comprimento especificado.
"""
    return string_1 + '-' * (length - len(string_1))


def print_matrix(string_1, string_2, matrix):
    """
    Imprime a matriz de pontuação com rótulos de sequência.

    Parâmetros:
    - string_1 (str): Rótulo da primeira sequência.
    - string_2 (str): Rótulo da segunda sequência.
    - matrix (List[List[int]]): Matriz de pontuação a ser impressa.
    """
    # Imprime rótulos da segunda sequência
    print(f'    {string_2}')
    
    # Itera sobre cada linha da matriz
    for i, row in enumerate(matrix):
        # Verifica se ainda há rótulos da primeira sequência
        if i < len(string_1): 
            row_str = ' '.join(map(str, row))
            print(f'{string_1[i]} {row_str}')  # Imprime rótulo da primeira sequência e valores da linha
        else:
            break

## "def **nw_ciclico**"
Após adaptar os algoritmos dos alinhamentos progressivos e matriz de distâncias, é necessário implementar um método para calcular as distâncias progressivas e agrupar as sequências de acordo com essas distâncias: **nw_ciclico**

In [111]:
def nw_ciclico(sequencias, score_subst, score_space):
    """
    Calcula a dissimilaridade entre todas as sequências usando o algoritmo de Needleman-Wunsch.

    Parâmetros:
    - sequencias (List[Tuple[str, str]]): Lista de tuplas contendo nomes e sequências.
    - score_subst (int): Pontuação para correspondência ou penalidade para substituição no algoritmo de Needleman-Wunsch.
    - score_space (int): Penalidade para espaços (inserção ou exclusão) no algoritmo de Needleman-Wunsch.

    Retorna:
    List[List[float]]: Matriz de dissimilaridade simétrica entre as sequências.
    """
    # Inicialização da matriz de dissimilaridade
    num_sequencias = len(sequencias)
    distancia_matriz = [[0.0] * num_sequencias for _ in range(num_sequencias)]

    # Iteração sobre combinações únicas de pares de sequências
    for i in range(num_sequencias):
        for j in range(i + 1, num_sequencias):
            nome_i, seq_i = sequencias[i]
            nome_j, seq_j = sequencias[j]

            try:
                # Alinhamento de sequências e cálculo do score usando Needleman-Wunsch
                aligned_seq_i, aligned_seq_j, score = needleman_wunsch(seq_i, seq_j, score_subst, score_space)
                
                # Cálculo da dissimilaridade
                dissimilaridade = 1.0 - (score / max(len(aligned_seq_i), len(aligned_seq_j)))

                # Preenchimento da matriz de dissimilaridade
                distancia_matriz[i][j] = dissimilaridade
                distancia_matriz[j][i] = dissimilaridade
                
            except Exception as e:
                # Tratamento de exceções para lidar com falhas no alinhamento
                print(f"Failed to align sequence {nome_i} and {nome_j}: {e}")

    return distancia_matriz

## **Árvore Filogenética**

In [112]:
from Bio import pairwise2
from Bio import Phylo
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio.Phylo.TreeConstruction import DistanceCalculator, DistanceTreeConstructor
from io import StringIO

def criar_arvore_filogenetica(distancia_matriz, sequencias):
    """
    Cria uma árvore filogenética usando a biblioteca Biopython com base nas sequências fornecidas.

    Parâmetros:
    - distancia_matriz (List[List[float]]): Matriz de dissimilaridade entre as sequências.
    - sequencias (List[Tuple[str, str]]): Lista de tuplas contendo nomes e sequências.

    A função utiliza a biblioteca Biopython para calcular e construir uma árvore filogenética com base nas dissimilaridades entre as sequências.
    """
    # Obter os nomes e sequências das entidades
    nomes = [nome for nome, _ in sequencias]

    # Calcular o comprimento máximo apenas uma vez
    max_len = len(max(sequencias, key=lambda x: len(x[1]))[1])

    # Criar uma instância de MultipleSeqAlignment com as sequências ajustadas para o mesmo comprimento
    msa = MultipleSeqAlignment([SeqRecord(Seq(aprimorar_seq(seq, max_len)), id=nome) for nome, seq in sequencias])

    # Calcular a matriz de distância usando a identidade como métrica
    calculator = DistanceCalculator('identity')
    dm         = calculator.get_distance(msa)

    # Construir a árvore filogenética usando o método UPGMA
    constructor = DistanceTreeConstructor()
    tree        = constructor.upgma(dm)

    # Imprimir a árvore filogenética
    return tree



A partir dos algoritmos modificados, nw_ciclico, e uma função para "desenhar" a árvore a partir da distância da matriz podemos criar dois tipo de árvores filogenéticas em python.

### Exemplo:

In [113]:
# Exemplo de uso com 2 sequências
s1 = 'ATCTCGGCTAAAC'
s2 = 'ACCGTATCTCGGTGCA'

# Lista de tuplas contendo nome e sequência, alterar estes parâmetros para o número de sequências a analisar
sequencias = [('s1', s1), ('s2', s2)]

# Parâmetros da substituição ou da adição de espaços, mais parâmetros podem ser incorporados no código e escolhidos aqui
score_subst = 2
score_space = -4

# Aplicar o algoritmo Needleman-Wunsch ciclico
distancia_matriz = nw_ciclico(sequencias, score_subst, score_space)

# Criar a 1ª árvore filogenética 
print("1ª árvore:", tree1)
tree1 = criar_arvore_filogenetica(distancia_matriz, sequencias)

# Nomes de sequências codificados substituídos pelos nomes reais das sequências na segunda geração da árvore
tree_str = "(({}:0.375, {}:0.375)Inner1:0.0)RootedTree;".format(sequencias[0][0], sequencias[1][0])

# Criar a 2ª árvore filogenética usando outro método
print("2ª árvore:")
tree2 = Phylo.read(StringIO(tree_str), "newick")

# Exibir a árvore em formato de texto
Phylo.draw_ascii(tree2)

1ª árvore: Tree(rooted=True)
    Clade(branch_length=0, name='Inner1')
        Clade(branch_length=0.375, name='s2')
        Clade(branch_length=0.375, name='s1')
2ª árvore:
  ___________________________________________________________________________ s1
_|
 |___________________________________________________________________________ s2



Os testes unitários vão verificar se todas as funções utilizadas neste capítulo estão a funcionar conforme o esperado.

In [115]:
import unittest
from io import StringIO
from Bio import Phylo


class TestSequenciaTools(unittest.TestCase):
    def test_fragmentos(self):
        result = needleman_wunsch('ATGCGTCGA', 'aagta', 2, -4)
        expected_result = (-9, ('ATGCGTCGA', 'A--AGT--A'))
        self.assertEqual(result, expected_result)

    def test_apenas_um_match(self):
        result = needleman_wunsch('GGCATGCG', 'A', 2, -4)
        expected_result = (-26, ('GGCATGCG', '---A----'))
        self.assertEqual(result, expected_result)

    def test_nao_dna(self):
        with self.assertRaises(AssertionError):
            needleman_wunsch('HGJK', 'ATC', 2, -4)

    def test_sequencia_vazia(self):   
        with self.assertRaises(AssertionError):
          needleman_wunsch('', ' ', 2, -4)

    def test_simbolos(self):
     with self.assertRaises(AssertionError):
            needleman_wunsch('A--GCTG--ACG', 'AGTG$%$CACG', 2, -4)

    def test_subfuncoes(self):
        result = needleman_wunsch('aacgt', 'AACGT', 2, -4)
        expected_result = (10, ('AACGT', 'AACGT'))
        self.assertEqual(result, expected_result)

    def test_aprimorar_seq(self):
        string = 'ATCTCGGCTAAAC'
        length = 20
        result = aprimorar_seq(string, length)
        expected_result = 'ATCTCGGCTAAAC-------'
        self.assertEqual(result, expected_result)

    def test_nw_ciclico(self):
        s1 = 'ATCTCGGCTAAAC'
        s2 = 'ACCGTATCTCGGTGCA'
        sequencias = [('s1', s1), ('s2', s2)]
        score_subst = 2
        score_space = -4
        distancia_matriz = nw_ciclico(sequencias, score_subst, score_space)

        # Verifica se a matriz de distância é simétrica
        for i in range(len(distancia_matriz)):
            for j in range(i + 1, len(distancia_matriz[i])):
                self.assertEqual(distancia_matriz[i][j], distancia_matriz[j][i])

    def test_criar_arvore_filogenetica(self):
        s1 = 'ATCTCGGCTAAAC'
        s2 = 'ACCGTATCTCGGTGCA'
        sequencias = [('s1', s1), ('s2', s2)]
        score_subst = 2
        score_space = -4
        distancia_matriz = nw_ciclico(sequencias, score_subst, score_space)

        # Chama a função para criar a árvore filogenética
        tree = criar_arvore_filogenetica(distancia_matriz, sequencias)

        # Verifica se a execução foi bem-sucedida (sem erros)
        self.assertIsNotNone(tree)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)


F..FF.FFF
FAIL: test_apenas_um_match (__main__.TestSequenciaTools.test_apenas_um_match)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\dases\AppData\Local\Temp\ipykernel_8748\2173975970.py", line 15, in test_apenas_um_match
    self.assertEqual(result, expected_result)
AssertionError: Tuples differ: ('ATGCG', 'A----', -26) != (-26, ('GGCATGCG', '---A----'))

First differing element 0:
'ATGCG'
-26

First tuple contains 1 additional elements.
First extra element 2:
-26

- ('ATGCG', 'A----', -26)
+ (-26, ('GGCATGCG', '---A----'))

FAIL: test_fragmentos (__main__.TestSequenciaTools.test_fragmentos)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\dases\AppData\Local\Temp\ipykernel_8748\2173975970.py", line 10, in test_fragmentos
    self.assertEqual(result, expected_result)
AssertionError: Tuples differ: ('GTCGA', 'aagta', -26) != (-9, ('ATGC